In [137]:
# import packages needed for analysis
import pandas as pd
import numpy as np

## 1.	Create a combined dataframe with Natwest and LBG income data for year 2020 full year  
### a.	Data sets  
#### i.	Natwest data on sheet '1.1 - Income Statement'  
#### ii.	LBG data  on sheet '35'

In [138]:
# read LBG excel file and save income data table to dataframe. Worksheet name provided in the question
# the correct columns and rows to extract are identified by looking at the correct sheet when opening the file
# blank rows that aren't part of the table are removed, and columns are renamed to something more useful

src_file1 = 'Lloyds Banking Group 2020 EOY.xlsx'

df1 = pd.read_excel(src_file1, sheet_name='35',header=4, usecols=[0,4])
df1 = df1.rename(columns={'Unnamed: 0': 'Measures',2020:'LBG'})
df1 = df1.dropna(subset=['Measures'])

In [139]:
# read Natwest excel file and save income data table to dataframe. Worksheet name provided in the question
# the correct columns and rows to extract are identified by looking at the correct sheet when opening the file
# blank rows that aren't part of the table are removed, and columns are renamed to something more useful
# some measures that will be needed later are renamed to match the other imported data set
# there is also an error in the Net Interest Income value as recorded in the excel file. The correct value can be determined from the quarterly values and updated accordingly

src_file2 = 'NatWest Group 2020 EOY .xlsx'

df2 = pd.read_excel(src_file2, sheet_name='1.1 - Income Statement',header=6, usecols=[1,18])
df2 = df2.rename(columns={'Unnamed: 1': 'Measures','£m.11':'Natwest'})
df2 = df2.dropna(subset=['Measures'])
df2 = df2.replace(['Total non-interest income','Operating expenses','Impairment losses','7#df749'],['Other income','Total operating expenses','Impairment',7749])

In [140]:
# the two data frames are joined on the names of the measures
joined_df =pd.merge(df1,df2, how="outer", on='Measures')

### b.	Extract the following lines into one dataframe  
#### i.	Net interest income (this is the amount of income the firm receives from interest payments)  
#### ii.	Total non-interest income / other income (this is income from other sources e.g. commission)  
#### iii.	Total income  
#### iv.	Operating expenses (this is the cost/expense of running the business)  
#### v.	Impairments (this is the amount of money the firm has lost due to loan defaults)  


In [141]:
# the measures listed in the question are extracted into a new dataframe
# it is noted that the LBG data has two rows for Total Income, one with Insurance Claims and one without. I have used the one labelled the same as the question
filtered_df = joined_df[joined_df['Measures'].isin(['Net interest income', 'Other income','Total income','Total operating expenses','Impairment'])]
print(filtered_df)

                    Measures    LBG Natwest
2        Net interest income  10749    7749
9               Other income  18418    3047
10              Total income  29167   10796
15  Total operating expenses  -9745   -7905
16                Impairment  -4155   -3242


#### c.	Create a new data frame containing all instances of the word ‘income’ and their respective values for 2020

In [147]:
# the question isn't clear which dataframe should be used, so I have used the one created in the previous question, but it can be easily amended if needed 
income_df = filtered_df[filtered_df['Measures'].str.contains("income")]
print(income_df)

               Measures    LBG Natwest
2   Net interest income  10749    7749
9          Other income  18418    3047
10         Total income  29167   10796


## 2.	Calculations (we want to check the data and create a new metric)  
### a.	For both firms  
#### i.	create a data validation rule = net interest income + non-interest income = total income and create a flag/exception if rule fails  
#### ii.	Calculate a new metric, profit before tax = total income - expenses - impairments  

In [143]:
# I have transposed the dataframe to make it easier to perform calculations on
calculations_df = filtered_df.set_index('Measures').T

In [144]:
# new columns are created to save the results from the new calculations, as defined in the question
# it should be noted that the expenses and impairments in both data sets are recorded as negative, so those values should be added to the income value rather than subtracted
calculations_df['Validation'] = calculations_df['Net interest income']+calculations_df['Other income']==calculations_df['Total income']
calculations_df['Profit before tax'] = calculations_df['Total income'] + calculations_df['Total operating expenses'] + calculations_df['Impairment']
calculations_df.head()

Measures,Net interest income,Other income,Total income,Total operating expenses,Impairment,Validation,Profit before tax
LBG,10749,18418,29167,-9745,-4155,True,15267
Natwest,7749,3047,10796,-7905,-3242,True,-351


## 3.	Modelling assumptions (We want to apply modelling assumptions to the data)  
#### i.	For Natwest, adjust the net interest income line up by 50%  
#### ii.	For LBG, adjust operating expenses line up by 75%

In [145]:
# the adjustments are performed in place in the dataframe
calculations_df.loc['Natwest','Net interest income'] = calculations_df.loc['Natwest','Net interest income'] *1.5
calculations_df.loc['LBG','Total operating expenses'] = calculations_df.loc['LBG','Total operating expenses'] *1.75

### b.	Recalculate profit before tax for both firms

In [146]:
# the new Profit before tax is recorded in a new column so it can be compared to the previous value
# Total income hasn't been updated in line with the change above to Net interest income, so the constituant parts are used instead
calculations_df['New profit before tax'] = calculations_df['Net interest income'] + calculations_df['Other income']+ calculations_df['Total operating expenses'] + calculations_df['Impairment']
calculations_df.head()

Measures,Net interest income,Other income,Total income,Total operating expenses,Impairment,Validation,Profit before tax,New profit before tax
LBG,10749,18418,29167,-17053.8,-4155,True,15267,7958.25
Natwest,11623.5,3047,10796,-7905,-3242,True,-351,3523.5


## 4.	Conclusion  
### a.	Determine which firm is the most profit before and after the adjustments and explain why using the data extracted for this test

Using the data extracted for this test, LBG has a greater pre-tax profit before (LBG: £15,267m vs Natwest: -£351m) and after (£7,958m vs £3,524m) the adjustments specified in the questions.
However, LBG has insurance claims within their Consolidate Income Statement which have a value of £14,041m. If these are included then LBG still has a greater pre-tax profit before adjustments (£1,226m vs -£351m), however Natwest's profits are greater post adjustment (-£6,083m vs £3,524m).

If this was a more generic paragraph to print, based on a repeatable script, then I would use f-strings to insert values and names from the tables produced based on set criteria. However, there are specific points that I wanted to highlight so have decided to use Markdown on this occasion.